### IMPORT DES BIBLIOTHÉQUES ET INITIALISATION DES OPTIONS

In [1]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import json
import sqlite3
import re
import time

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
load_dotenv()

True

### Réccupération des données de Festival de France avec l'API de DATA.GOUV

In [ ]:
dataset_id = "festivals-global-festivals-_-pl"
api_key = os.getenv("API_KEY")  
url = f"https://data.culture.gouv.fr/api/v2/catalog/datasets/{dataset_id}/exports/json"

headers = {
    'X-API-KEY': api_key
}

response = requests.get(url, headers=headers)
print("Code d'état:", response.status_code)
print("Contenu de la réponse", response.text)

if response.status_code == 200:
    try:
        # Charger les données JSON depuis la réponse de l'API
        data = response.json()

        # Afficher les données JSON formatées
        print(json.dumps(data, indent=1))

        # Convertir les données en un DataFrame pandas
        df = pd.DataFrame(data)

        # Enregistrer le DataFrame au format CSV pour éviter les limitations de l'utilisation de l'api 
        csv_filename = 'festivals_data.csv'
        df.to_csv(csv_filename, index=False)
    except json.JSONDecodeError as e:
        print("Erreur de décodage JSON:", e)
else:
    print("Échec de la requête. Code d'état:", response.status_code)

### EXPLORATION DU JEU DE DONNÉES

In [119]:
df = pd.read_csv("festivals_data.csv")


In [43]:
df.head(5)

,nom_du_festival,envergure_territoriale,region_principale_de_deroulement,departement_principal_de_deroulement,commune_principale_de_deroulement,code_postal_de_la_commune_principale_de_deroulement,code_insee_commune,code_insee_epci_collage_en_valeur,libelle_epci_collage_en_valeur,numero_de_voie,type_de_voie_rue_avenue_boulevard_etc,nom_de_la_voie,adresse_postale,complement_d_adresse_facultatif,site_internet_du_festival,adresse_e_mail,decennie_de_creation_du_festival,annee_de_creation_du_festival,discipline_dominante,sous_categorie_spectacle_vivant,sous_categorie_musique,sous_categorie_musique_cnm,sous_categorie_cinema_et_audiovisuel,sous_categorie_arts_visuels_et_arts_numeriques,sous_categorie_livre_et_litterature,periode_principale_de_deroulement_du_festival,identifiant_agence_a,identifiant,geocodage_xy,identifiant_cnm
0,Gamerz,Départementale,Provence-Alpes-Côte d'Azur,Bouches-du-Rhône,Aix-en-Provence,13090,13001,200054807,Métropole d'Aix-Marseille-Provence,1,Place,Victor schoelcher,1 Place Victor schoelcher,Patio du bois de l'aune,http://www.lab-gamerz.com/site/,contact@lab-gamerz.com,De 2000 à 2009,2006,"Arts visuels, arts numériques",NaN,NaN,NaN,NaN,Arts numériques,NaN,Après-saison (6 septembre - 31 décembre),NaN,FEST_13001_49,"{'lon': 5.39857444582, 'lat': 43.5360708378}",NaN
1,Corsiclassic - festival de musique de chambre,NaN,Corse,Corse-du-Sud,Ajaccio,20000,2A004,242010056,CA du Pays Ajaccien,NaN,Rue,Forcioli conti,Rue Forcioli conti,Cathédrale d'Ajaccio,helloasso.com/associations/corsiclassic,Corsiclassic@gmail.com ou bartoli.jeanne@wanad...,2010 et après,2016,Musique,NaN,Musique classique,NaN,NaN,NaN,NaN,Après-saison (6 septembre - 31 décembre),NaN,FEST_2A004_85,"{'lon': 8.70132275974, 'lat': 41.9347926638}",NaN
2,Racines de ciel,NaN,Corse,Corse-du-Sud,Ajaccio,20000,2A004,242010056,CA du Pays Ajaccien,50-52,Rue,Du cardinal Fesch,50-52 Rue Du cardinal Fesch,NaN,https://www.facebook.com/RacinesDeCielAjaccio/,Viagrenelle@gmail.com,De 2000 à 2009,2009,"Livre, littérature",NaN,NaN,NaN,NaN,NaN,"Fiction (roman, théâtre, etc.), Non-fiction (d...",Avant-saison (1er janvier - 20 juin),NaN,FEST_2A004_96,"{'lon': 8.70132275974, 'lat': 41.9347926638}",NaN
3,Le grand bivouac,Intercommunale,Auvergne-Rhône-Alpes,Savoie,Albertville,73200,73011,200068997,CA Arlysère,88 bis,Rue,de la République,88 bis Rue de la République,Le Grand Bivouac d'Albertville,https://www.grandbivouac.com/fr,communication@grandbivouac.org,De 2000 à 2009,2002,"Cinéma, audiovisuel",NaN,NaN,NaN,Films documentaires courts et moyens métrages,NaN,NaN,Après-saison (6 septembre - 31 décembre),NaN,FEST_73011_104,"{'lon': 6.40460338643, 'lat': 45.6683987277}",NaN
4,Festival Acthea,NaN,Occitanie,Tarn,Albi,81000,81004,248100737,CA de l'Albigeois (C2A),NaN,Allée,Des Sciences,Allée Des Sciences,Ecoles Des Mines D'albi - Campus Jarlard,https://www.mines-albi.fr/acthea-17e-edition-d...,NaN,NaN,NaN,Spectacle vivant,Théâtre,NaN,NaN,NaN,NaN,NaN,Avant-saison (1er janvier - 20 juin),NaN,FEST_81004_113,"{'lon': 2.14686328555, 'lat': 43.9258213622}",NaN


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7282 entries, 0 to 7281
Data columns (total 30 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   nom_du_festival                                      7282 non-null   object 
 1   envergure_territoriale                               1898 non-null   object 
 2   region_principale_de_deroulement                     7282 non-null   object 
 3   departement_principal_de_deroulement                 7256 non-null   object 
 4   commune_principale_de_deroulement                    7256 non-null   object 
 5   code_postal_de_la_commune_principale_de_deroulement  7265 non-null   object 
 6   code_insee_commune                                   7258 non-null   object 
 7   code_insee_epci_collage_en_valeur                    7246 non-null   object 
 8   libelle_epci_collage_en_valeur                       7246 non-null  

In [26]:
df.columns.tolist()

['nom_du_festival',
 'envergure_territoriale',
 'region_principale_de_deroulement',
 'departement_principal_de_deroulement',
 'commune_principale_de_deroulement',
 'code_postal_de_la_commune_principale_de_deroulement',
 'code_insee_commune',
 'code_insee_epci_collage_en_valeur',
 'libelle_epci_collage_en_valeur',
 'numero_de_voie',
 'type_de_voie_rue_avenue_boulevard_etc',
 'nom_de_la_voie',
 'adresse_postale',
 'complement_d_adresse_facultatif',
 'site_internet_du_festival',
 'adresse_e_mail',
 'decennie_de_creation_du_festival',
 'annee_de_creation_du_festival',
 'discipline_dominante',
 'sous_categorie_spectacle_vivant',
 'sous_categorie_musique',
 'sous_categorie_musique_cnm',
 'sous_categorie_cinema_et_audiovisuel',
 'sous_categorie_arts_visuels_et_arts_numeriques',
 'sous_categorie_livre_et_litterature',
 'periode_principale_de_deroulement_du_festival',
 'identifiant_agence_a',
 'identifiant',
 'geocodage_xy',
 'identifiant_cnm']

### SUPPRESSION DES COLONNES INUTILES

In [120]:
df = df.drop([
 'envergure_territoriale',
 'code_insee_epci_collage_en_valeur',
 'libelle_epci_collage_en_valeur',
 'adresse_e_mail',
 'decennie_de_creation_du_festival',
 'identifiant_agence_a',
 'identifiant',
 'adresse_postale',
 'identifiant_cnm',], axis=1)

In [46]:
df.columns

Index(['nom_du_festival', 'region_principale_de_deroulement',
       'departement_principal_de_deroulement',
       'commune_principale_de_deroulement',
       'code_postal_de_la_commune_principale_de_deroulement',
       'code_insee_commune', 'numero_de_voie',
       'type_de_voie_rue_avenue_boulevard_etc', 'nom_de_la_voie',
       'complement_d_adresse_facultatif', 'site_internet_du_festival',
       'annee_de_creation_du_festival', 'discipline_dominante',
       'sous_categorie_spectacle_vivant', 'sous_categorie_musique',
       'sous_categorie_musique_cnm', 'sous_categorie_cinema_et_audiovisuel',
       'sous_categorie_arts_visuels_et_arts_numeriques',
       'sous_categorie_livre_et_litterature',
       'periode_principale_de_deroulement_du_festival', 'geocodage_xy'],
      dtype='object')

### CREATION D'UN NOUVEAU JEU DE DONNÉES 

In [121]:
# Renommer certaines colonnes
df_renomme = df.rename(columns={
    'nom_du_festival': 'Nom_Festival',
    'region_principale_de_deroulement': 'Region',
    'departement_principal_de_deroulement': 'Departement',
    'commune_principale_de_deroulement': 'Commune',
    'code_insee_commune': 'Code_INSEE',
    'site_internet_du_festival': 'Site_Internet',
    'annee_de_creation_du_festival': 'Annee_Creation',
    'discipline_dominante': 'Discipline_Principale',
    'periode_principale_de_deroulement_du_festival': 'Periode',
    'geocodage_xy': 'Geocode'
})

# Convertir toutes les valeurs en chaînes de caractères avant de les combiner
colonnes_adresse = ['numero_de_voie', 'type_de_voie_rue_avenue_boulevard_etc', 
                    'nom_de_la_voie', 'code_postal_de_la_commune_principale_de_deroulement', 
                    'complement_d_adresse_facultatif']

df_renomme[colonnes_adresse] = df_renomme[colonnes_adresse].fillna('').astype(str)

# Créer la colonne 'Adresse_Postale'
df_renomme['Adresse_Postale'] = df_renomme[colonnes_adresse].agg(' '.join, axis=1)

# Regrouper les sous-catégories en une seule colonne 'Sous_Categorie'
colonnes_sous_categorie = ['sous_categorie_spectacle_vivant', 'sous_categorie_musique',
                           'sous_categorie_musique_cnm', 'sous_categorie_cinema_et_audiovisuel',
                           'sous_categorie_arts_visuels_et_arts_numeriques',
                           'sous_categorie_livre_et_litterature']

df_renomme[colonnes_sous_categorie] = df_renomme[colonnes_sous_categorie].fillna('').astype(str)

df_renomme['Sous_Categorie'] = df_renomme[colonnes_sous_categorie].agg(' '.join, axis=1)

# Sélectionner uniquement les colonnes nécessaires pour le nouveau DataFrame
colonnes_selectionnees = [
    'Nom_Festival', 'Region', 'Departement', 'Commune', 'Adresse_Postale', 
    'Code_INSEE', 'Site_Internet', 'Annee_Creation', 
    'Discipline_Principale', 'Sous_Categorie', 'Periode', 'Geocode'
]


df_festival = df_renomme[colonnes_selectionnees]



### TRAITEMENT DES COLONNES

**Colonnes GeoCode**

In [ ]:
# Création de colonne longitude et latitude et suppressopn de la colonne Geocode
df_festival['Geocode'] = df_festival['Geocode'].str.replace("'", "\"")

geocode_str = df_festival['Geocode'][0]
geocode_dict = json.loads(geocode_str)

df_festival['Geocode'] = df_festival['Geocode'].apply(lambda x: json.loads(x) if pd.notnull(x) else None)

df_festival['longitude'] = df_festival['Geocode'].apply(lambda x: x['lon'] if x is not None else None)
df_festival['latitude'] = df_festival['Geocode'].apply(lambda x: x['lat'] if x is not None else None)

df_festival.drop('Geocode', axis=1, inplace=True)

In [109]:
df_festival.tail(5)

,Nom_Festival,Region,Departement,Commune,Adresse_Postale,Code_INSEE,Site_Internet,Annee_Creation,Discipline_Principale,Sous_Catégorie,Periode,longitude,latitude
7277,Vitrollywood,Provence-Alpes-Côte d'Azur,Bouches-Du-Rhône,Vitrolles,13127,13117,facebook.com/vatoslocosvideo,NaN,"Cinéma, audiovisuel",,Saison (21 juin - 5 septembre),5.263578,43.449783
7278,Entente Nocturne,Île-de-France,Val-de-Marne,Vitry-sur-Seine,94400,94081,http://lekilowatt.fr/agenda/entente-nocturne/,2018-01-01 00:00:00,Musique,Musiques actuelles,Après-saison (6 septembre - 31 décembre),2.394127,48.788283
7279,Festival des cultures du monde,Auvergne-Rhône-Alpes,Isère,Voiron,45 Avenue de Paviot 38500,38563,https://www.salmorencvoironculturesdumonde.com...,1983,Spectacle vivant,Danse,Saison (21 juin - 5 septembre),5.582403,45.379172
7280,Buisso'Nied,Grand Est,Moselle,Volmerange-lès-Boulay,57320,57730,https://www.facebook.com/profile.php?id=100027...,1994,Spectacle vivant,Conte,Après-saison (6 septembre - 31 décembre),6.449554,49.170169
7281,Festival le Mois Kreyol,Guadeloupe,Guadeloupe,NaN,,NaN,https://lemoiskreyol.fr/,2017,Pluridisciplinaire,,Période variable selon les territoires,NaN,NaN


**Colonne Anne_Creation**

In [ ]:
# nettoyer la date en gardant les 4 premiers caractères si la date est au format "AAAA-MM-JJ HH:MM:SS"

def clean_year(date_str):
    if isinstance(date_str, str):  
        if re.match(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}", date_str):
            return date_str[:4]  # Garder uniquement l'année
        elif re.match(r"\d{4}", date_str):
            return date_str
    return date_str 

# Uniformisation de la colonne Annee_Creation en  appliquant la fonction uniquement aux valeurs au format "YYYY-MM-DD HH:MM:SS"
df_festival['Annee_Creation'] = df_festival['Annee_Creation'].apply(clean_year)


In [ ]:
df_festival['Annee_Creation'].unique()

In [ ]:
# Suppression de toutes les caractéres non numérique de la colonne Anne_Creation 
def supprimer_non_numerique(year_str):
    if isinstance(year_str, str):
        return re.sub(r'\D', '', year_str)
    return None

df_festival['Annee_Creation'] = df_festival['Annee_Creation'].apply(supprimer_non_numerique)



In [ ]:
df_festival['Annee_Creation'].unique()

In [ ]:
# Remplacer "31564" par None  
df_festival['Annee_Creation'] = df_festival['Annee_Creation'].replace('31564', None)

def nettoyage_annee(year_str):
    if isinstance(year_str, str):
        # Pour les lignes avec plus de 4 caractères, garder uniquement les 4 premiers caractères
        if len(year_str) > 4:
            return year_str[:4]
        # Pour les lignes avec 2 caractères, les remplacer par "2024 - XX"
        elif len(year_str) == 2:
            return f"2024 - {year_str}"
        else:
            return year_str
    return year_str

df_festival['Annee_Creation'] = df_festival['Annee_Creation'].apply(nettoyage_annee)

In [ ]:
def get_real_year(year_str):
    if isinstance(year_str, str):
        # Vérifier si la chaîne commence par "2024 - "
        if year_str.startswith("2024 - "):
            try:
                # Extraire XX de la chaîne
                xx = int(year_str.split("2024 - ")[1])
                # Calculer l'année réelle
                real_year = 2024 - xx
                return str(real_year)
            except ValueError:
                return year_str  # En cas d'erreur, retourner la chaîne d'origine
    return year_str

# Appliquer la fonction pour obtenir l'année réelle à la colonne 'Annee_Creation' de df_festival
df_festival['Annee_Creation'] = df_festival['Annee_Creation'].apply(get_real_year)


In [138]:
df_festival['Annee_Creation'].unique()

array(['2006', '2016', '2009', '2002', None, '2014', '1983', '1998',
       '1996', '1980', '2004', '1997', '2015', '2000', '1999', '2010',
       '1985', '2005', '2001', '1986', '1982', '1948', '2011', '1992',
       '1989', '2019', '2018', '2003', '2007', '1993', '2017', '2012',
       '2013', '1979', '1971', '1988', '2021', '1978', '1990', '1995',
       '2008', '2020', '1946', '1966', '1987', '1984', '1972', '1991',
       '1950', '1994', '1974', '1981', '1970', '1976', '1977', '1962',
       '2022', '1973', '1968', '1969', '1967', '620', '1934', '9', '1920',
       '1958', '1930', '1964', '8', '1905', '5', '1975', '1952', '1961',
       '1923', '1604', '1956', '1874', '3', '1221', '1965', '1957',
       '1954', '1947', '1960', '1944', '1951', '1949', '1935', '1959',
       '520', '1869'], dtype=object)

In [ ]:
def update_years(year_str):
    if isinstance(year_str, str):
        if len(year_str) == 1 or len(year_str) == 3:
            try:
                # Obtenir le premier caractère de la chaîne
                first_char = int(year_str[0])
                # Calculer l'année réelle
                real_year = 2024 - first_char
                return f"2024 - {real_year}"
            except ValueError:
                return year_str  # En cas d'erreur, retourner la chaîne d'origine
    return year_str

# Appliquer la fonction pour mettre à jour les années à la colonne 'Annee_Creation' de df_festival
df_festival['Annee_Creation'] = df_festival['Annee_Creation'].apply(update_years)


In [ ]:
df_festival['Annee_Creation'] = df_festival['Annee_Creation'].str.replace('2024 - ', '')

In [ ]:
df_festival['Annee_Creation'].unique()

In [ ]:
df_festival['Annee_Creation'] = df_festival['Annee_Creation'].replace(['1221', '1604'], None) ## a suppriemr

**Colonne Sous-Categorie**

In [ ]:
# df_festival["Sous_Categorie"].unique().tolist()

In [160]:
def clean_sous_categorie(value):
    # Supprimer les numéros suivis d'un tiret et d'un espace
    value = re.sub(r'\d+\s*-\s*', '', value)
    # Supprimer les espaces au début et à la fin
    value = value.strip()
    # Remplacer les valeurs vides par 'Inconnu' (vous pouvez choisir une autre valeur si nécessaire)
    if not value:
        return 'Inconnu'
    return value


df_festival['Sous_Categorie'] = df_festival['Sous_Categorie'].apply(clean_sous_categorie)


/tmp/ipykernel_9173/2264096737.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_festival['Sous_Catégorie'] = df_festival['Sous_Catégorie'].apply(clean_sous_categorie)


In [ ]:
# df_festival['Sous_Categorie'].unique().tolist()

**Colonne Period**
 - Creation d'une nouvelle colonne 'Categorie_Periode'

In [164]:
def categoriser_periode(periode):
    if pd.isna(periode):
        return "Inconnu"
    periode = periode.lower()
    if 'avant-saison' in periode:
        return 'Avant-saison'
    elif 'après-saison' in periode:
        return 'Après-saison'
    elif 'saison' in periode:
        return 'Saison'
    elif any(mois in periode for mois in ['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'septembre', 'octobre', 'novembre', 'décembre']):
        return 'Mois spécifique'
    else:
        return 'Variable'

df_festival['Categorie_Periode'] = df_festival['Periode'].apply(categoriser_periode)


/tmp/ipykernel_9173/1557041798.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_festival['Categorie_Periode'] = df_festival['Periode'].apply(categoriser_periode)


**ADRESSE**

In [169]:
def get_address_from_coordinates(lat, lon):
    try:
        # Construisez l'URL de requête pour Nominatim
        url = f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json"

        # Ajoutez un délai avant d'effectuer la requête pour respecter la limite de requête par seconde
        time.sleep(1)

        # Effectuez la requête GET à Nominatim
        response = requests.get(url)

        # Vérifiez si la requête a réussi
        if response.status_code == 200:
            # Analysez la réponse JSON
            json_data = response.json()

            # Extrayez l'adresse de la réponse JSON
            if "address" in json_data:
                address = json_data["address"]

                # Construisez l'adresse complète
                full_address = ", ".join(filter(None, [
                    address.get("road", ""),
                    address.get("quarter", ""),
                    address.get("city", ""),
                    address.get("municipality", ""),
                    address.get("county", ""),
                    address.get("state", ""),
                    address.get("region", ""),
                    address.get("postcode", "")
                ]))

                return full_address
    except Exception as e:
        print(f"Error getting address: {e}")
    return ""

df_festival['Adresse_Complete'] = df_festival.apply(lambda row: get_address_from_coordinates(row['latitude'], row['longitude']) if not pd.isna(row['latitude']) and not pd.isna(row['longitude']) else "", axis=1)
